# Final Scoring Function

In [11]:
from shingle import *
from graph import *
import numpy as np

In [7]:
portuguese = open('data/portuguese/port.txt', "r+")
wordlist = {}
for line in portuguese:
    word = line.rstrip()
    wordlist[word] = two_ends(word, 2)

In [12]:
train = open('data/portuguese/portuguese_train_mrr.txt', "r+")
pairs = []
for line in train:
    word = line.rstrip().split('____')
    pairs.append((word[0], word[1]))
data = [(p[0], two_ends(p[0],2), p[1]) for p in pairs]
avgdl = np.mean([len(s[1]) for s in data])
print("Average is", avgdl)

Average is 9.136666666666667


In [13]:
port_ro_cog = dict()
port_ro_noncog = dict()

def init_dict():
    cogs = open("data/portuguese/graph_cognates_freq.txt", "r+")
    for line in cogs:
        splits = line.rstrip().split()
        port_ro_cog[(splits[1], splits[2])] = int(splits[0])
    cogs.close()
    noncogs = open("data/portuguese/graph_noncognates_freq.txt", "r+")
    for line in noncogs:
        splits = line.rstrip().split()
        port_ro_noncog[(splits[1], splits[2])] = int(splits[0])
    noncogs.close()

init_dict()

In [46]:
def pi(source, target, k = 1.5):
    query = two_ends(source, 2) #Your query
    document = two_ends(target, 2) #Your document
    qd = common_elements(query, document) # q cap d
    first = uncommon_elements(query, qd) # q - (q cap d)
    second = uncommon_elements(document, qd) # d - (q cap d)
    graph = graph_model(first,second)
    res = 0 # sum the frequencies in the dictionary
    for i in graph:
        if i in port_ro_cog:
            res += port_ro_cog[i] * k
    return res / len(graph)

In [25]:
pi("aspirat", "aspirar")

1.5

In [26]:
shingles = 470751

def smooth(intersection, document, mu):
    smooth = []
    for word in intersection:
        prob = 1.0 + np.divide(document.count(word), mu * frequencies[word] / shingles)
        smooth.append(np.log10(prob))
    smooth = np.array(smooth)
    return smooth

def dirichlet(query, document, mu = 100.0):
    intersection = [word for word in document if word in query] # intersection
    add = len(query) * np.log10(np.divide(mu, mu + len(document)))
    score = np.dot(tf(intersection, query), smooth(intersection , document, mu)) + add
    return score

def tf(intersection, document):
    tf = [document.count(word) for word in intersection]
    return np.array(tf)

In [27]:
# Initialize counts
frequencies = {}
text = open("data/portuguese/two_ends.txt", "r+")
for line in text:
    word = line.strip().split(' ')
    frequencies[word[0]] = float(word[1])   

In [38]:
def score(query, document, alpha = 0.5):
    return alpha * pi(query, document) + (1.0 - alpha) * dirichlet(two_ends(query, 2), two_ends(document, 2))

In [39]:
print(pi("aspirat", "aspirar"))
print(dirichlet(two_ends("aspirat", 2), two_ends("aspirar", 2)))

1.5
5.1532544571197025


In [40]:
print(score("aspirat", "aspirar"))

3.3266272285598513


In [48]:
for source in data:
    score_list = []
    for key in wordlist:
        score_list.append((key, score(key, source[0])))
    score_list = sorted(score_list, key=lambda x: x[1], reverse=True)
    
    for i in range(5):
        print(score_list[i])

('parosmia', 4.166831082625134)
('parolo', 1.8582312759732964)
('heroísmos', 1.5841405494757694)
('passeie', 1.4544081730068719)
('parolos', 1.419937607422565)
('médiuns', 2.709144654545853)
('imune', 1.8815913033974492)
('impune', 1.8589298140041204)
('auto-imune', 1.7682838564308057)
('decocção', 1.7643142065607795)
('luxação', 2.996564293494911)
('médiuns', 2.7248658290595826)
('luxo', 2.4697662993827945)
('luxos', 2.386132893866619)
('luxuriante', 2.088775105180787)


KeyboardInterrupt: 